## Анализ базы данных сервиса для чтения книг по подписке


**Входные данные**

База данных содержит информацию о книгах, издательствах, авторах, пользовательских оценках и обзорах.

1) таблица books cодержит данные о книгах:
     - book_id — идентификатор книги;
     - author_id — идентификатор автора;
     - title — название книги;
     - num_pages — количество страниц;
     - publication_date — дата публикации книги;
     - publisher_id — идентификатор издателя.

2) таблица authors cодержит данные об авторах:
     - author_id — идентификатор автора;
     - author — имя автора

3) таблица publishers cодержит данные об издательствах:
     - publisher_id — идентификатор издательства;
     - publisher — название издательства;

4) таблица ratings cодержит данные о пользовательских оценках книг:
     - rating_id — идентификатор оценки;
     - book_id — идентификатор книги;
     - username — имя пользователя, оставившего оценку;
     - rating — оценка книги.

5) таблица reviews cодержит данные о пользовательских обзорах на книги:
     - review_id — идентификатор обзора;
     - book_id — идентификатор книги;
     - username — имя пользователя, написавшего обзор;
     - text— текст обзора.

## Подключение к БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


## Исследование таблиц

In [8]:
tables = ['books','authors','publishers','ratings','reviews']

for i in range(5):
    query = '''
        SELECT *
        FROM {}
        LIMIT 5
        '''.format(tables[i])
    display(tables[i])
    display(pd.io.sql.read_sql(query, con = engine))

'books'

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


'authors'

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


'publishers'

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


'ratings'

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


'reviews'

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задания

In [9]:
# Посчитайте, сколько книг вышло после 1 января 2000 года
query = '''

SELECT 
    COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 2000 года было издано 819 книг.

In [11]:
#Для каждой книги посчитайте количество обзоров и среднюю оценку
query = '''

SELECT 
    b.title,
    COUNT(DISTINCT rev.review_id) AS total_review,
    AVG(rat.rating) AS avg_rating
FROM books AS b
LEFT JOIN ratings AS rat on b.book_id = rat.book_id
LEFT JOIN reviews AS rev on b.book_id = rev.book_id
GROUP BY b.title
ORDER BY total_review DESC
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,total_review,avg_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


Вывели информацию для 999 книг. Количетсво обзоров от 1120 до 0, при этом заметим, что средняя оценка не коррелирует с количеством отзывов. 

In [12]:
#Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры
query = '''
WITH
bigbook AS (
SELECT
    publisher_id,
    book_id
FROM books
WHERE num_pages > 50
)

SELECT 
    p.publisher_id,
    p.publisher
FROM bigbook AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
GROUP BY p.publisher_id
ORDER BY COUNT (b.book_id) DESC
LIMIT 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,212,Penguin Books


Наиболее широко представлены книги издательства Penguin Books

In [13]:
#Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
query = '''

WITH 

highrat AS (
SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
)


SELECT  
       a.author, 
       AVG(r.rating)
FROM books AS b
JOIN highrat ON b.book_id = highrat.book_id
JOIN authors AS a ON b.author_id = a.author_id
JOIN ratings AS r ON highrat.book_id = r.book_id

GROUP BY a.author
ORDER BY AVG(r.rating) DESC
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Самые популярный автор это Джоан Роулинг. Ожидаемо, узник азкабана. Ср. балл 4.41

In [14]:
#Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок
query = '''
WITH
big_rat AS (
SELECT 
    username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48
)

SELECT 
    COUNT(DISTINCT rev.text) / COUNT(DISTINCT rev.username)
FROM ratings AS r
JOIN reviews AS rev ON r.username = rev.username
JOIN big_rat ON r.username = big_rat.username

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,?column?
0,24


Пользователи, поставившее больше 48 оценок, в  среднем оставляли 24 отзыва.